In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

## PO

In [6]:
species_po = pd.read_csv('../../../Data_TER/GLC24-PO-metadata-train.csv')
species_po['speciesId'] = species_po['speciesId'].astype(int)

In [7]:
po_bio_ave = pd.read_csv('../../../Data_TER/GLC24-PO-train-bioclimatic-average.csv')
po_elev = pd.read_csv('../../../Data_TER/GLC24-PO-train-elevation.csv')
po_human = pd.read_csv('../../../Data_TER/GLC24-PO-train-human-footprint.csv')
po_land = pd.read_csv('../../../Data_TER/GLC24-PO-train-landcover.csv')
po_soilgrid = pd.read_csv('../../../Data_TER/GLC24-PO-train-soilgrids.csv')

In [8]:
abio_po = po_bio_ave.merge(po_elev, on='surveyId')\
        .merge(po_human, on='surveyId')\
                .merge(po_land, on='surveyId')\
                    .merge(po_soilgrid, on='surveyId')
abio_po = abio_po.replace([np.inf], np.nan)
abio_po = abio_po.replace([-np.inf], np.nan)
abio_po = abio_po.fillna(abio_po.mean())

In [9]:
print("Espèces : ","\n",species_po.head(), "\n", "\n")
print("Données abio : ","\n",abio_po.head())

Espèces :  
    year  month   day       lat       lon  geoUncertaintyInM taxonRank  \
0  2019    5.0   5.0  43.74605  1.573057                6.0   SPECIES   
1  2021    3.0  17.0  42.12559  0.314948                5.0   SPECIES   
2  2021    6.0   5.0  48.29520 -0.934518               24.9   SPECIES   
3  2021    6.0   9.0  53.63367 -2.644535                8.0   SPECIES   
4  2021    4.0   1.0  49.79471  7.925086               15.0   SPECIES   

         date  dayOfYear  speciesId  surveyId  
0  2019-05-05        125       3383         1  
1  2021-03-17         76       1152         2  
2  2021-06-05        156       6772         3  
3  2021-06-09        160       3318         4  
4  2021-04-01         91       3374         5   
 

Données abio :  
    surveyId  Bio1  Bio2  Bio3  Bio4  Bio5  Bio6  Bio7  Bio8  Bio9  ...  \
0         1  2865    88     3  5756  2999  2752   247  2888  2938  ...   
1         2  2865   116     3  6844  3028  2721   307  2869  2795  ...   
2         3  284

## PA

In [10]:
train_bio_ave = pd.read_csv('../../Data/Data_2024/Environnemental_Data/train/GLC24-PA-train-bioclimatic-average.csv')
train_elev = pd.read_csv('../../Data/Data_2024/Environnemental_Data/train/GLC24-PA-train-elevation.csv')
train_human = pd.read_csv('../../Data/Data_2024/Environnemental_Data/train/GLC24-PA-train-human-footprint.csv')
train_land = pd.read_csv('../../Data/Data_2024/Environnemental_Data/train/GLC24-PA-train-landcover.csv')
train_soilgrid = pd.read_csv('../../Data/Data_2024/Environnemental_Data/train/GLC24-PA-train-soilgrids.csv')

In [11]:
abio_pa = train_bio_ave.merge(train_elev, on='surveyId')\
        .merge(train_human, on='surveyId')\
                .merge(train_land, on='surveyId')\
                    .merge(train_soilgrid, on='surveyId')

In [12]:
abio_pa = abio_pa.replace([np.inf], np.nan)
abio_pa = abio_pa.replace([-np.inf], np.nan)
abio_pa.iloc[:, 1:47] = abio_pa.iloc[:, 1:47].fillna(abio_pa.iloc[:, 1:47].mean())

In [13]:
species_pa = pd.read_csv('../../Data/Data_2024/GLC24-PA-metadata-train.csv')
species_pa['speciesId'] = species_pa['speciesId'].astype(int)

## PA ET PO

In [14]:
species_po2 = species_po.iloc[:,9:]
species_pa2 = species_pa.iloc[:,7:]

In [15]:
species_all = pd.concat([species_po2,species_pa2],axis=0)
species_all

,speciesId,surveyId
0,3383,1
1,1152,2
2,6772,3
3,3318,4
4,3374,5
...,...,...
1483632,53,3919655
1483633,5071,3919655
1483634,1889,3919655
1483635,10600,3919655


In [16]:
species_group = species_all.groupby('speciesId')['surveyId'].agg(list).reset_index()
species_group

,speciesId,surveyId
0,0,"[35695, 44444, 57757, 92365, 96301, 175296, 18..."
1,1,[1842284]
2,2,"[66890, 82411, 83701, 89358, 97349, 132719, 15..."
3,3,[1974081]
4,4,"[2226187, 62976, 124250, 172626, 298514, 33446..."
...,...,...
10353,11248,"[394179, 1548873, 1660413, 1810688, 2167375, 2..."
10354,11250,"[81665, 204058, 305059, 318984, 1064949, 13599..."
10355,11252,"[26072, 310032, 528646, 681664, 1008167, 14557..."
10356,11253,"[166928, 189851, 204294, 443138, 500287, 53266..."


Lors de l'ajout des 0 ou 1 pour chaque espèce, il faut rajouter les surveyId de po où l'espèce est présente sinon une espèce uniquement dans les po renverra une absence totale

In [82]:
#Pour l'espèce 0 :
pd.concat([abio_po['surveyId'][abio_po['surveyId'].isin(species_group.iloc[1,1])],abio_pa['surveyId']])


1807963    1842284
0              212
1              222
2              243
3              324
            ...   
88982      3919553
88983      3919592
88984      3919620
88985      3919640
88986      3919655
Name: surveyId, Length: 88988, dtype: int64

In [17]:
presence = []

for i in range(5):
    survey_id = species_group.iloc[i,1]
    species_id = species_group.iloc[i,0]
    # Pour chaque espèce on stock 0 ou 1 pour chaque surveyId
    presence_espece = pd.concat([abio_po['surveyId'][abio_po['surveyId'].isin(species_group.iloc[i,1])],abio_pa['surveyId']])
    # On rassemble les présences de toutes les espèces dans une liste
    presence.append((str(species_id), presence_espece))
    print(i)

0
1
2
3
4


In [85]:
presence[1]

('1',
 1807963    1842284
 0              212
 1              222
 2              243
 3              324
             ...   
 88982      3919553
 88983      3919592
 88984      3919620
 88985      3919640
 88986      3919655
 Name: surveyId, Length: 88988, dtype: int64)

## Random Forest pour compléter PO

In [19]:
#pd.concat([abio_po[abio_po['surveyId'].isin(species_group.iloc[1,1])].iloc[:,1:],abio_pa.iloc[:,1:]])
#abio_po[~abio_po['surveyId'].isin(species_group.iloc[0,1])].iloc[:,1:]

,Bio1,Bio2,Bio3,Bio4,Bio5,Bio6,Bio7,Bio8,Bio9,Bio10,...,LandCover,Soilgrid-bdod,Soilgrid-cec,Soilgrid-cfvo,Soilgrid-clay,Soilgrid-nitrogen,Soilgrid-phh2o,Soilgrid-sand,Soilgrid-silt,Soilgrid-soc
0,2865,88,3,5756,2999,2752,247,2888,2938,2941,...,12.0,145.000000,212.000000,130.000000,303.000000,134.000000,73.000000,306.000000,389.000000,122.000000
1,2865,116,3,6844,3028,2721,307,2869,2795,2958,...,17.0,134.000000,213.000000,189.000000,302.000000,184.000000,74.000000,294.000000,403.000000,223.000000
2,2842,71,3,4857,2954,2752,202,2814,2906,2906,...,9.0,127.000000,211.000000,109.000000,187.000000,257.000000,59.000000,132.000000,681.000000,301.000000
3,2829,59,3,4108,2918,2747,171,2807,2843,2883,...,9.0,103.000000,212.000000,123.000000,216.000000,466.000000,56.000000,429.000000,355.000000,1044.000000
4,2833,76,3,6506,2970,2718,252,2901,2791,2918,...,14.0,138.000000,195.000000,131.000000,310.000000,198.000000,73.000000,208.000000,480.000000,208.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3846449,2833,65,3,5280,2947,2737,210,2802,2793,2901,...,13.0,121.883834,224.604377,135.598598,225.354681,324.975966,61.316752,375.847043,397.341461,488.828339
3846450,2797,63,2,6874,2935,2677,258,2799,2745,2888,...,1.0,107.000000,252.000000,118.000000,108.000000,315.000000,47.000000,445.000000,445.000000,1101.000000
3846451,2861,91,3,6157,3008,2741,267,2864,2944,2944,...,5.0,118.000000,201.000000,187.000000,232.000000,210.000000,68.000000,318.000000,449.000000,311.000000
3846452,2827,69,2,5949,2954,2722,232,2762,2785,2904,...,5.0,121.000000,200.000000,121.000000,251.000000,281.000000,55.000000,260.000000,487.000000,395.000000


In [20]:
for i in range (5):
    X_train = pd.concat([abio_po[abio_po['surveyId'].isin(species_group.iloc[i,1])].iloc[:,1:],abio_pa.iloc[:,1:]])
    X_train.to_csv("../../../TER_RF/train/xtrain_species"+str(i)+".csv", index=False, sep=',')
    X_test = abio_po[~abio_po['surveyId'].isin(species_group.iloc[i,1])].iloc[:,1:]
    X_test.to_csv("../../../TER_RF/test/xtest_species"+str(i)+".csv", index=False, sep=',')


Il faut ajouter au Xtrain les valeurs abiotique de présence de PO

In [23]:
%%time
predict_po={}
model = RandomForestClassifier(n_estimators=5)
# len(species_group)-1

for i in range(4):
    X = pd.read_csv('../../../TER_RF/train/xtrain_species'+str(i)+'.csv')
    Y = presence[i][1]
    model.fit(X, Y)
    X_test = pd.read_csv('../../../TER_RF/test/xtest_species'+str(i)+'.csv')
    Y_pred = model.predict_proba(X_test)[:,1]
    species_name = presence[i][0] 
    predict_po[species_name] = Y_pred
    print(i)

In [18]:
predict_po={}
model = RandomForestClassifier(n_estimators=5)
X = pd.read_csv('../../../TER_RF/train/xtrain_species'+str(0)+'.csv')
Y = presence[0][1]
model.fit(X, Y)

In [ ]:
X_test = pd.read_csv('../../../TER_RF/test/xtest_species'+str(0)+'.csv')   
Y_pred = model.predict_proba(X_test)[:,1]
species_name = presence[0][0] 
predict_po[species_name] = Y_pred